# Семинар 3: Немного о np.reshape

## Двумерные матрицы и reshape

Посмотрим, что происходит с обычными матрицами, когда мы делаем reshape

In [ ]:
import numpy as np
from copy import deepcopy

Рассмотрим матрицу и её транспонированную. Посмотрим на их флаги: какая contigiocity у них по умолчанию?

In [ ]:
A = np.array([
    [0,1,2],
    [3,4,5]
])
A.flags, np.transpose(A).flags

(  C_CONTIGUOUS : True
   F_CONTIGUOUS : False
   OWNDATA : True
   WRITEABLE : True
   ALIGNED : True
   WRITEBACKIFCOPY : False,
   C_CONTIGUOUS : False
   F_CONTIGUOUS : True
   OWNDATA : False
   WRITEABLE : True
   ALIGNED : True
   WRITEBACKIFCOPY : False)

In [ ]:
A.shape

(2, 3)

Давайте порешейпим нашу матрицу в другой размер с разными порядками. Посмотрим, как именно размещаются элементы внутри, и какие флаги contigiocity выставляются. Бонус: что происходит с owndata?

In [ ]:
A_c = deepcopy(A)
print(f'{A_c = }')
f_reshaped_A_c = A_c.reshape((3, 2), order='f')
c_reshaped_A_c = A_c.reshape((3, 2), order='c')
print('===F-style===')
print(f'{f_reshaped_A_c = }')
print(f_reshaped_A_c.flags)
print('===C-style===')
print(f'{c_reshaped_A_c = }')
print(c_reshaped_A_c.flags)

A_c = array([[0, 1, 2],
       [3, 4, 5]])
===F-style===
f_reshaped_A_c = array([[0, 4],
       [3, 2],
       [1, 5]])
  C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

===C-style===
c_reshaped_A_c = array([[0, 1],
       [2, 3],
       [4, 5]])
  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False



In [ ]:
A_c = deepcopy(A)
A_c[:,1][0]=5
print(f'{A_c = }')
print(A_c.flags)

print('===[:,1]===')
print(A[:,1].flags)
print('===[1,:]===')
print(A[1,:].flags)
print('===to fortran [:,1]===')
print(np.asfortranarray(deepcopy(A))[:,1].flags)
print('===to fortran [1,:]===')
print(np.asfortranarray(deepcopy(A))[1, :].flags)

A_c = array([[0, 5, 2],
       [3, 4, 5]])
  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

===[:,1]===
  C_CONTIGUOUS : False
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

===[1,:]===
  C_CONTIGUOUS : True
  F_CONTIGUOUS : True
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

===to fortran [:,1]===
  C_CONTIGUOUS : True
  F_CONTIGUOUS : True
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

===to fortran [1,:]===
  C_CONTIGUOUS : False
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False



In [ ]:
m, n = 3000, 1000
big_A = np.random.randn(m, n)
print(f'{big_A.flags.c_contiguous = }')

print('====Reshape===')
res_c = %timeit -o -q big_A.reshape(2, -1)
print(f'c-style: {res_c.average * 1e3:.{4}f}', " ms")
res_c = %timeit -o -q big_A.reshape(-1, 2)
print(f'c-style: {res_c.average * 1e3:.{4}f}', " ms")

res_f = %timeit -o -q big_A.reshape((2, -1), order='f')
print(f'f-style: {res_f.average * 1e3:.{4}f}', " ms")
res_f = %timeit -o -q big_A.reshape((-1, 2), order='f')
print(f'f-style: {res_f.average * 1e3:.{4}f}', " ms")


print('====Transpose===')

res_c = %timeit -o -q big_A.T
print(f'c-style: {res_c.average * 1e3:.{4}f}', " ms")

bbig_A = np.asfortranarray(deepcopy(big_A))
res_f = %timeit -o -q bbig_A.T
print(f'f-style: {res_f.average * 1e3:.{4}f}', " ms")

res_non = %timeit -o -q bbig_A[:m - 1, :n - 1].T # lost contig.
print(f'non-style: {res_non.average * 1e3:.{4}f}', " ms")

big_A.flags.c_contiguous = True
====Reshape===
c-style: 0.0005  ms
c-style: 0.0005  ms
f-style: 10.2419  ms
f-style: 9.1515  ms
====Transpose===
c-style: 0.0003  ms
f-style: 0.0001  ms
non-style: 0.0004  ms


## Тензоры и reshape

Перейдем к многомерному случаю.

In [ ]:
A = np.array([          # первый индекс: внешний (по "матрицам")
    [                   # второй индекс: средний (по "строкам матрицы")
        [0, 1],         # последний индекс: "внутри строки"
        [2, 3],
    ],
    [
        [4, 5],
        [6, 7]
    ],
    [
        [8, 9],
        [10, 11]
    ],
])
A = A.transpose(1, 2, 0) # теперь первый индекс --- строки матриц,
                         # второй --- столбы,
                         # третий --- выбор матрицы

In [ ]:
A[:, :, 0], A[:, :, 1], A[:, :, 2],

(array([[0, 1],
        [2, 3]]),
 array([[4, 5],
        [6, 7]]),
 array([[ 8,  9],
        [10, 11]]))

C-порядок в первую очередь меняет **последний** индекс и идет в переборе "с конца", в то время как F-порядок меняет первый и идет с начала.

In [ ]:
A.reshape(2, -1)

array([[ 0,  4,  8,  1,  5,  9],
       [ 2,  6, 10,  3,  7, 11]])

In [ ]:
A.reshape((2, -1), order='f')

array([[ 0,  1,  4,  5,  8,  9],
       [ 2,  3,  6,  7, 10, 11]])

Давайте напишем несложную программу с циклами, которая выводит элементы тензора так, будто они в C- или F-порядке

In [ ]:
def print_C_order(A):
    shape = A.shape     # A[i,j,k]
    for i in range(shape[0]):
        for j in range(shape[1]):
            for k in range(shape[2]):
                print(A[i,j,k], end = " ")

In [ ]:
print_C_order(A)

0 4 8 1 5 9 2 6 10 3 7 11 

In [ ]:
def print_F_order(A):
    shape = A.shape     # A[i,j,k]
    for k in range(shape[2]):
        for j in range(shape[1]):
            for i in range(shape[0]):
                print(A[i,j,k], end = " ")

In [ ]:
print_F_order(A)

0 2 1 3 4 6 5 7 8 10 9 11 